In [22]:
from rdkit import Chem
from rdkit.Chem import AllChem
import numpy as np
import pandas as pd
# SMILES 문자열을 분자 객체로 변환




In [23]:
def finger(df):
    fingerprint_list = []
    
    for smile in df['SMILES']:
        mol = Chem.MolFromSmiles(f"{smile}")

        # 분자 객체에서 ECFP 지문 생성
        fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=512)
        # 지문을 배열로 변환
        arr = np.array(fp)
        fingerprint_list.append(arr)

    fingerprint_df = pd.DataFrame(fingerprint_list)

    
    return fingerprint_df

In [24]:
train_df = pd.read_csv('./origin_data/train.csv')

finger_df = finger(train_df)

finger_train_df = pd.concat([train_df,finger_df],axis=1)

finger_train_df.to_csv('./data/finger_r2b512_train.csv')

In [25]:
test_df = pd.read_csv('./origin_data/test.csv')

finger_df = finger(test_df)

finger_test_df = pd.concat([test_df,finger_df],axis=1)

finger_test_df.to_csv('./data/finger_r2b512_test.csv')